In [1]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

Problem 2

In [2]:
#Create MILP model object
mymodel = glp.Solver('FTBC', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [3]:
inf = mymodel.infinity()

         # name: supply capacity (thousand tons), fixed cost (thousand $), unit shipping cost ($) to each RDC
source = {'Canton': (5000,10000 , [ 2.34, 1.28, 0.6, 1.6]), 
          'Dayton': (5000,12000 , [ 0.54, 0.71, 2.13, 1.5]), 
          'Lima':   (5000,10000 , [ 1.29, 0.92, 1.73, 0.79]),    
         }
          
        # name: demand quantity required (thousand tons)
demand = {'Cincinnati': 2100, 'Columbus': 2000, 'Cleveland': 2000,'Toledo': 600}    # Scranton Atlanta Reno

mymodel.Objective().SetMinimization()    # minimize total plant and transportation cost

In [4]:
# create binary Plant selection variables and a dictionary to access them
select = dict()
for s in source:
    select[s] = mymodel.IntVar(0,1,s)
    (b, f, coeff_lst ) = source[s]
    mymodel.Objective().SetCoefficient(select[s], f)
print(select)
lpt.print_model(mymodel)

{'Canton': Canton, 'Dayton': Dayton, 'Lima': Lima}
Variables:
Canton, Dayton, Lima 

minimize: 10000.0*Canton + 12000.0*Dayton + 10000.0*Lima 

Subject To:

Bounds:
Canton in [0, 1]
Dayton in [0, 1]
Lima in [0, 1]


In [5]:
# create shipment variables from Plants to RDCs and a dictionary to access them
ship = dict()
for s in source:
    ship[s] = dict()
    (b, f, coeff_lst) = source[s]
    for (d,c) in zip(demand,coeff_lst):
        ship[s][d] = mymodel.NumVar(0, inf, s + '.' + d)
        mymodel.Objective().SetCoefficient(ship[s][d] ,c )
print(ship)

{'Canton': {'Cincinnati': Canton.Cincinnati, 'Columbus': Canton.Columbus, 'Cleveland': Canton.Cleveland, 'Toledo': Canton.Toledo}, 'Dayton': {'Cincinnati': Dayton.Cincinnati, 'Columbus': Dayton.Columbus, 'Cleveland': Dayton.Cleveland, 'Toledo': Dayton.Toledo}, 'Lima': {'Cincinnati': Lima.Cincinnati, 'Columbus': Lima.Columbus, 'Cleveland': Lima.Cleveland, 'Toledo': Lima.Toledo}}


In [6]:
# create source/supply constraints
for s in source:
    (b, f, coeff_lst) = source[s]
    constr = mymodel.Constraint(-inf ,0 ,s )
    constr.SetCoefficient(select[s] , -b)
    for d in demand:
        constr.SetCoefficient(ship[s][d], 1)
print(source)

{'Canton': (5000, 10000, [2.34, 1.28, 0.6, 1.6]), 'Dayton': (5000, 12000, [0.54, 0.71, 2.13, 1.5]), 'Lima': (5000, 10000, [1.29, 0.92, 1.73, 0.79])}


In [7]:
# create demand constraints
for d in demand:
    lb = ub = demand[d]
    constr = mymodel.Constraint(lb ,ub ,d )
    for s in source:
        constr.SetCoefficient(ship[s][d] , 1)
print(demand)

{'Cincinnati': 2100, 'Columbus': 2000, 'Cleveland': 2000, 'Toledo': 600}


In [8]:
lpt.print_model(mymodel)

Variables:
Canton, Dayton, Lima, Canton.Cincinnati, Canton.Columbus, Canton.Cleveland, Canton.Toledo, Dayton.Cincinnati, Dayton.Columbus, Dayton.Cleveland, Dayton.Toledo, Lima.Cincinnati, Lima.Columbus, Lima.Cleveland, Lima.Toledo 

minimize: 10000.0*Canton + 12000.0*Dayton + 10000.0*Lima + 2.34*Canton.Cincinnati + 1.28*Canton.Columbus + 0.6*Canton.Cleveland + 1.6*Canton.Toledo + 0.54*Dayton.Cincinnati + 0.71*Dayton.Columbus + 2.13*Dayton.Cleveland + 1.5*Dayton.Toledo + 1.29*Lima.Cincinnati + 0.92*Lima.Columbus + 1.73*Lima.Cleveland + 0.79*Lima.Toledo 

Subject To:
Canton: - 5000.0*Canton + 1.0*Canton.Cincinnati + 1.0*Canton.Columbus + 1.0*Canton.Cleveland + 1.0*Canton.Toledo <= 0.0
Dayton: - 5000.0*Dayton + 1.0*Dayton.Cincinnati + 1.0*Dayton.Columbus + 1.0*Dayton.Cleveland + 1.0*Dayton.Toledo <= 0.0
Lima: - 5000.0*Lima + 1.0*Lima.Cincinnati + 1.0*Lima.Columbus + 1.0*Lima.Cleveland + 1.0*Lima.Toledo <= 0.0
Cincinnati: 1.0*Canton.Cincinnati + 1.0*Dayton.Cincinnati + 1.0*Lima.Cincinnati 

In [9]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f thousand' % mymodel.Objective().Value())
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%7s = %5.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 26223.00 thousand
 Canton =  1.00
   Lima =  1.00
Canton.Cleveland = 2000.00
Lima.Cincinnati = 2100.00
Lima.Columbus = 2000.00
Lima.Toledo = 600.00


In [10]:
intial_cost = 26223.00

Second Best solution

In [11]:
# store coefficients and RHS for cut constraint

a = dict()
M = 0
for s in source:
    if select[s].solution_value() == 1:
        a[s] =  1
        M = M + 1
    else:    # solution_value == 0
        a[s] =  -1
print('M =', M, ', a=', a)

#add cut constraint
cut_constr = mymodel.Constraint(-inf ,M - 1 , 'Cut' + str([a[s] for s in a]))
for s in source:
    cut_constr.SetCoefficient(select[s] , a[s])
    
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f thousand' % mymodel.Objective().Value())
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%7s = %5.2f' % (v.name(),v.solution_value()))
            
# relax cut constraint
# cut_constr.SetBounds(-mymodel.infinity(),mymodel.infinity())

M = 2 , a= {'Canton': 1, 'Dayton': -1, 'Lima': 1}
Solution Status = 0
Optimal Value = 26654.00 thousand
 Canton =  1.00
 Dayton =  1.00
Canton.Cleveland = 2000.00
Dayton.Cincinnati = 2100.00
Dayton.Columbus = 2000.00
Dayton.Toledo = 600.00


In [12]:
second_best = 26654.00
cost_difference = second_best - intial_cost

In [13]:
print("The difference in cost for this alternate solution is: {}".format(cost_difference))

The difference in cost for this alternate solution is: 431.0
